In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import pandas as pd
import os

In [2]:
#import revenue data with BQ
# Initialize a BigQuery client
client = bigquery.Client()
gcp_project = os.environ['GCP_PROJECT']
bq_dataset = os.environ['BQ_DATASET']

# full orders BQ query and df
query = f"""
SELECT * FROM `{gcp_project}.{bq_dataset}.orders_full`
"""
df = client.query(query).to_dataframe()

#Dropping unnecessary columns, grouping by "date", summing "item_price" to get daily revenues
df = pd.DataFrame(df.groupby(by="date")["item_price"].sum()/100)
df = df.rename(columns={"date": "ds", "item_price": "y"})
df["ds"] = df.index
df = df.reset_index(drop=True)
df = df[["ds","y"]]
#turning the ds (date) column into datetim
df['ds']=pd.to_datetime(df['ds'])
#Dropping outliers
df = df[df["y"]>=60]
df = df[df["y"]<=2300]
df = df.reset_index(drop=True)

#remove the duplicates
#DELETE duplicates = df['ds'].duplicated()

# weather BQ query and df
query = f"""
SELECT * FROM `{gcp_project}.{bq_dataset}.weather`
"""
weather_df = client.query(query).to_dataframe()

#drop some columns
weather_df = weather_df.drop(columns=["dt","timezone","city_name","lat","lon","sea_level","grnd_level","weather_icon","rain_3h","snow_3h"])
#rename
weather_df = weather_df.rename(columns={"dt_iso":"ds","rain_1h":"rain","clouds_all":"clouds"})
# convert the timestamp column to a datetime object with timezone information
weather_df['ds'] = weather_df['ds'].str[:19]
weather_df["ds"] = pd.to_datetime(weather_df["ds"])
# Filter the rows between 8 am and 10 pm
df_filtered = weather_df[(weather_df['ds'].dt.hour >= 8) & (weather_df['ds'].dt.hour <= 22)]
#replace the nan value with 0 in whole dataset
df_filtered = df_filtered.fillna(0)

#group by he rain by sum
df_filtered['sum_rain'] = df_filtered.groupby(pd.Grouper(key='ds', freq='D'))['rain'].transform('sum')
df_filtered
# Drop all rows where the time component of 'column1' is not 12:00:00
df_filtered = weather_df[weather_df['ds'].dt.time == pd.to_datetime('12:00:00').time()]
# convert the timestamp column to a datetime object
df_filtered['ds'] = pd.to_datetime(df_filtered['ds'])
# remove the hour from the ds column
df_filtered['ds'] = df_filtered['ds'].dt.normalize()
# convert the ds column back to datetime format without hour
df_filtered['ds'] = pd.to_datetime(df_filtered['ds']).dt.date
df_filtered["ds"] = pd.to_datetime(df_filtered["ds"])
#drop the drop_duplicates
df_filtered = df_filtered.drop_duplicates()
#merge sum weather data after sum rain with df
merged_df = pd.merge(df, df_filtered, on='ds', how='left')
#drop_duplicates
merged_df.drop_duplicates(subset='ds', inplace=True)

# holidays BQ query and df
query = f"""
SELECT * FROM `{gcp_project}.{bq_dataset}.holidays`
"""
df_holiday = client.query(query).to_dataframe()
#df_holiday = df_holiday.reset_index()
#df_holiday.columns = df_holiday.iloc[0]
# drop the first row, which is now redundant
# df_holiday = df_holiday.drop(0)
df_holiday['ds'] = pd.to_datetime(df_holiday['ds'])
df_holiday.info()
#merge holiday to df
merged_df  = pd.merge(merged_df,df_holiday,on="ds",how="left")
#DELETE merged_df = merged_df_h

# inflation BQ query and df
query = f"""
SELECT * FROM `{gcp_project}.{bq_dataset}.inflation`
"""
df_inflation_rate = client.query(query).to_dataframe()
df_inflation_rate['ds'] = pd.to_datetime(df_inflation_rate['ds'])
#merge inflation to df
merged_df= pd.merge(merged_df, df_inflation_rate, how='left', left_on='ds', right_on='ds')

# consumption climate query and df
query = f"""
SELECT * FROM `{gcp_project}.{bq_dataset}.consumption_climate`
"""
df_consumption_climate = client.query(query).to_dataframe()
df_consumption_climate['ds'] = pd.to_datetime(df_consumption_climate['ds'])
#merge
merged_df = pd.merge(merged_df, df_consumption_climate,on="ds",how="left")

#covid
merged_df['cov_lock'] = ((merged_df['ds'] >= pd.to_datetime("2020-03-22")) & (merged_df['ds'] <= pd.to_datetime("2020-04-21"))) | ((merged_df['ds'] >= pd.to_datetime("2020-12-16")) & (merged_df['ds'] <= pd.to_datetime("2021-01-09"))) | ((merged_df['ds'] >= pd.to_datetime("2020-04-22")) & (merged_df['ds'] <= pd.to_datetime("2020-05-03"))) | ((merged_df['ds'] >= pd.to_datetime("2020-10-10")) & (merged_df['ds'] <= pd.to_datetime("2020-12-15"))) | ((merged_df['ds'] >= pd.to_datetime("2021-03-31")) & (merged_df['ds'] <= pd.to_datetime("2021-05-18"))) | ((merged_df['ds'] >= pd.to_datetime("2021-11-02")) & (merged_df['ds'] <= pd.to_datetime("2021-12-09")))
merged_df['cov_lock'] = merged_df['cov_lock'].astype(int)

# Berlin Unemployment Mitte and Mitte Mitte query and df
query = f"""
SELECT * FROM `{gcp_project}.{bq_dataset}.berlin_unemployment`
"""
df_unemp_ber = client.query(query).to_dataframe()
# fom object to datetype
df_unemp_ber['Date'] = pd.to_datetime(df_unemp_ber['Date'])
#df_unemp_ber[['unemp_Berlin_Mitte', 'unemp_Berlin_Mitte_Mitte']] = df_unemp_ber[['unemp_Berlin_Mitte', 'unemp_Berlin_Mitte_Mitte']].apply(lambda x: x.str.replace(',', '.'))
#df_unemp_ber = df_unemp_ber.astype({"unemp_Berlin_Mitte": float, "unemp_Berlin_Mitte_Mitte": float})
df_unemp_ber[['unemp_Berlin_Mitte', 'unemp_Berlin_Mitte_Mitte']] = df_unemp_ber[['unemp_Berlin_Mitte', 'unemp_Berlin_Mitte_Mitte']].div(100)
df_unemp_ber.set_index('Date', inplace=True)
df_unemp_ber = df_unemp_ber.resample('D').ffill()
merged_df = pd.merge(merged_df, df_unemp_ber, how='left', left_on='ds', right_on='Date')

#Fill NaNs and strip whitespaces
merged_df = merged_df.fillna(0)
merged_df = merged_df.rename(columns=lambda x: str(x).strip())
#Drop unnecessary columns based on correlation or co-correlation
merged_df.drop(['visibility', 'dew_point', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'wind_gust',
        'snow_1h', 'weather_id', 'weather_main', 'weather_description', 'unemp_Berlin_Mitte_Mitte'], axis=1, inplace=True)

/tmp/ipykernel_22001/3402223984.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ds'] = pd.to_datetime(df_filtered['ds'])
/tmp/ipykernel_22001/3402223984.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ds'] = df_filtered['ds'].dt.normalize()
/tmp/ipykernel_22001/3402223984.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ds       850 non-null    datetime64[ns]
 1   Holiday  850 non-null    Int64         
dtypes: Int64(1), datetime64[ns](1)
memory usage: 14.2 KB


/tmp/ipykernel_22001/3402223984.py:109: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_unemp_ber['Date'] = pd.to_datetime(df_unemp_ber['Date'])


In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703 entries, 0 to 1702
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ds                   1703 non-null   datetime64[ns]
 1   y                    1703 non-null   Float64       
 2   temp                 1703 non-null   float64       
 3   humidity             1703 non-null   Int64         
 4   wind_speed           1703 non-null   float64       
 5   wind_deg             1703 non-null   Int64         
 6   rain                 1703 non-null   float64       
 7   clouds               1703 non-null   Int64         
 8   Holiday              1703 non-null   Int64         
 9   inflation_rate       1703 non-null   float64       
 10  consumption_climate  1703 non-null   float64       
 11  cov_lock             1703 non-null   int64         
 12  unemp_Berlin_Mitte   1703 non-null   float64       
dtypes: Float64(1), Int64(4), datetime

In [5]:
merged_df.dtypes

ds                     datetime64[ns]
y                             Float64
temp                          float64
humidity                        Int64
wind_speed                    float64
wind_deg                        Int64
rain                          float64
clouds                          Int64
Holiday                         Int64
inflation_rate                float64
consumption_climate           float64
cov_lock                        int64
unemp_Berlin_Mitte            float64
dtype: object